In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType

# --- Setup ---
dbutils.widgets.text("run_date", "2025-12-09")
source_table = "final_project.silver.crimes_silver"

print(f"Reading from Silver: {source_table}")
df_silver = spark.read.table(source_table)

# ==============================================================================
# TABLE 1: Daily Trends (For Time Series & General Trends)
# ==============================================================================
print("Building: gold_daily_trends...")
# Aggregation: Crimes per day and category
df_daily = df_silver.groupBy("crime_date", "Crime_Category") \
    .agg(
        F.count("*").alias("total_crimes"),
        F.sum(F.when(F.col("Arrest") == "true", 1).otherwise(0)).alias("total_arrests"),
        F.sum(F.when(F.col("Domestic") == "true", 1).otherwise(0)).alias("total_domestic")
    ) \
    .orderBy("crime_date")

df_daily.write.format("delta").mode("overwrite").saveAsTable("final_project.gold.daily_trends")
print("Saved: final_project.gold.daily_trends")


# ==============================================================================
# TABLE 2: District Stats (For Clustering, Spatial Analysis & Correlations)
# ==============================================================================
print("Building: gold_district_stats...")
# Aggregation: Profile of each Police District
df_district = df_silver.groupBy("District") \
    .agg(
        F.count("*").alias("crime_count"),
        F.mean("Latitude").alias("center_lat"),
        F.mean("Longitude").alias("center_long"),
        # Calculate arrest rate per district
        (F.sum(F.when(F.col("Arrest") == "true", 1).otherwise(0)) / F.count("*")).alias("arrest_rate"),
        # Counts by category
        F.sum(F.when(F.col("Crime_Category") == "Violent", 1).otherwise(0)).alias("violent_crimes_count"),
        F.sum(F.when(F.col("Crime_Category") == "Property", 1).otherwise(0)).alias("property_crimes_count")
    )

df_district.write.format("delta").mode("overwrite").saveAsTable("final_project.gold.district_stats")
print("Saved: final_project.gold.district_stats")


# ==============================================================================
# TABLE 3: Hourly Patterns (For Heatmaps & Pattern Discovery)
# ==============================================================================
print("Building: gold_hourly_heatmap...")
# Aggregation: Day of Week + Hour (Matrix 7x24)
df_heatmap = df_silver.groupBy("day_of_week", "crime_hour", "Crime_Category") \
    .agg(F.count("*").alias("incident_count")) \
    .orderBy("day_of_week", "crime_hour")

df_heatmap.write.format("delta").mode("overwrite").saveAsTable("final_project.gold.hourly_heatmap")
print("Saved: final_project.gold.hourly_heatmap")


# ==============================================================================
# TABLE 4: ML Features (For Classification & Regression)
# ==============================================================================
print("Building: gold_ml_features...")
# Preparation: Select only numerical/categorical features for modeling
# Convert Boolean strings (true/false) to Integers (1/0)
df_ml = df_silver.select(
    "District",
    "Latitude", "Longitude",
    "day_of_week", "crime_hour", "crime_month",
    "hour_sin", "hour_cos", "month_sin", "month_cos",
    "is_weekend",
    # Target variables (Labels)
    F.when(F.col("Arrest") == "true", 1).otherwise(0).alias("label_arrest"),
    F.col("Crime_Category").alias("label_category"),
    F.col("Primary_Type") 
)

df_ml.write.format("delta").mode("overwrite").saveAsTable("final_project.gold.ml_features")
print("Saved: final_project.gold.ml_features")


# ==============================================================================
# TABLE 5: Geo Incidents (For Map Visualization - Deliverable 2 & 3)
# ==============================================================================
print("Building: gold_geo_incidents...")
# Selection: Clean coordinates and display info for scatter plots
df_geo = df_silver.select(
    "crime_date",
    "crime_time",
    "Latitude", 
    "Longitude",
    "Primary_Type",   # For popup info
    "Crime_Category", # For color coding (Violent vs Property)
    "Arrest",
    "District"
).filter(
    (F.col("Latitude").isNotNull()) & (F.col("Longitude").isNotNull())
)

df_geo.write.format("delta").mode("overwrite").saveAsTable("final_project.gold.geo_incidents")
print("Saved: final_project.gold.geo_incidents")

print("\nAll Gold tables created successfully.")